# Classifier
For the first stage of this project, using trained data from nexto 2v2 using our observation builders, we create a model to predict what Nexto would do given our inputs. This is essentially model distillation.

In [27]:
import pickle
import os
from tqdm.notebook import tqdm
dir = "data"

In [28]:
def loadPKL(name):
    with open(os.path.join(dir, name), 'rb') as f:
        return [(x.astype('float32'), y.astype('float32'))for x,y in pickle.load(f)]
def loadAll():
    data = []
    files = [x for x in os.listdir(dir) if not x.startswith('.')]
    for f in tqdm(files):
        data+=loadPKL(f)
    return data

# Pytorch section

### Dataset

In [29]:
import numpy as np
import torch


from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

from torch import Tensor
from torch.nn import Linear
from torch.nn import SELU
from torch.nn import Tanh
from torch.nn import Module
from torch.nn import Dropout

from torch.nn import MSELoss
from torch.optim import Adam

In [30]:
class NextoDataset(Dataset):
    def __init__(self):
        data = loadAll()
        self.X = []
        self.Y = []
        for x,y in data:
            self.X.append(x)
            self.Y.append(y)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, i):
        return [self.X[i], self.Y[i]]
    def get_splits(self, n_test = .2):
        test_size = round(n_test*len(self.X))
        train_size = len(self.X) - test_size
        return random_split(self, [train_size, test_size])

### Model

In [34]:
class Net(Module):
    # define model elements
    def __init__(self):
        super().__init__()
        # input to first hidden layer
        n_inputs = 119
        
        self.dropout1 = Dropout(0.2)
        self.dropout2 = Dropout(0.2)
        
        self.hidden1 = Linear(n_inputs, 256)
        self.act1 = SELU()

        self.hidden2 = Linear(256, 256)
        self.act2 = SELU()

        self.hidden3 = Linear(256, 128)
        self.act3 = SELU()

        self.hidden4 = Linear(128, 64)
        self.act4 = SELU()
        
        # third hidden layer and output
        self.hidden5 = Linear(64, 8)
        self.act5 = Tanh()

    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        
        X = self.dropout1(X)
        
        # third hidden layer and output
        X = self.hidden3(X)
        X = self.act3(X)
        
        X = self.dropout2(X)
        
        X = self.hidden4(X)
        X = self.act4(X)
        
        X = self.hidden5(X)
        X = self.act5(X)
        return X

In [35]:
Net()

Net(
  (dropout1): Dropout(p=0.2, inplace=False)
  (dropout2): Dropout(p=0.2, inplace=False)
  (hidden1): Linear(in_features=119, out_features=256, bias=True)
  (act1): SELU()
  (hidden2): Linear(in_features=256, out_features=256, bias=True)
  (act2): SELU()
  (hidden3): Linear(in_features=256, out_features=128, bias=True)
  (act3): SELU()
  (hidden4): Linear(in_features=128, out_features=64, bias=True)
  (act4): SELU()
  (hidden5): Linear(in_features=64, out_features=8, bias=True)
  (act5): Tanh()
)

### Utility functions

In [36]:
def prepare_data(batch_size):
    # load the dataset
    dataset = NextoDataset()
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=batch_size, shuffle=True)
    test_dl = DataLoader(test, batch_size=batch_size, shuffle=False)
    return train_dl, test_dl

In [37]:
def train_model(train_dl, valid_dl, model,epochs = 10):
    if torch.cuda.is_available():
        model = model.cuda()
    min_valid_loss = np.inf
    # define the optimization
    criterion = MSELoss()
    optimizer = Adam(model.parameters())
    with tqdm(total=len(train_dl)) as bar:
        for e in range(epochs):
            train_loss = 0.0
            model.train()     # Optional when not using Model Specific layer
#             bar.reset()
            for data, labels in train_dl:
#                 bar.update(1)
                if torch.cuda.is_available():
                    data, labels = data.cuda(), labels.cuda()

                optimizer.zero_grad()
                target = model(data)
                loss = criterion(target,labels)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            valid_loss = 0.0
            model.eval()     # Optional when not using Model Specific layer
            for data, labels in valid_dl:
                if torch.cuda.is_available():
                    data, labels = data.cuda(), labels.cuda()

                target = model(data)
                loss = criterion(target,labels)
                valid_loss = loss.item() * data.size(0)

            print(f'Epoch {e+1} \tTraining Loss: {train_loss / len(train_dl)} \t Validation Loss: {valid_loss / len(valid_dl)}')
            if min_valid_loss > valid_loss:
                print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
                min_valid_loss = valid_loss
                torch.save(model.state_dict(), f'models/{valid_loss/len(valid_dl)*1000:.6f}.pth')


In [38]:
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

# Training Loop

In [39]:
train_dl, test_dl = prepare_data(batch_size=128)
print(len(train_dl.dataset), len(test_dl.dataset))

  0%|          | 0/431 [00:00<?, ?it/s]

345145 86286


In [40]:
model = Net()
paths = [float(x[:-4]) for x in os.listdir('models') if not x.startswith('.')]
best = min(paths)

model.load_state_dict(torch.load(f'models/{best:.6f}.pth'))

ValueError: min() arg is an empty sequence

In [42]:
model

Net(
  (dropout1): Dropout(p=0.2, inplace=False)
  (dropout2): Dropout(p=0.2, inplace=False)
  (hidden1): Linear(in_features=119, out_features=256, bias=True)
  (act1): SELU()
  (hidden2): Linear(in_features=256, out_features=256, bias=True)
  (act2): SELU()
  (hidden3): Linear(in_features=256, out_features=128, bias=True)
  (act3): SELU()
  (hidden4): Linear(in_features=128, out_features=64, bias=True)
  (act4): SELU()
  (hidden5): Linear(in_features=64, out_features=8, bias=True)
  (act5): Tanh()
)

In [ ]:
# train the model
train_model(train_dl, test_dl, model, epochs = 10000)

  0%|          | 0/2697 [00:00<?, ?it/s]

Epoch 1 	Training Loss: 0.2594438068151562 	 Validation Loss: 0.0063584986439457645
Validation Loss Decreased(inf--->4.291987) 	 Saving The Model
Epoch 2 	Training Loss: 0.25402330611872154 	 Validation Loss: 0.007201846705542671
Epoch 3 	Training Loss: 0.25169311951698087 	 Validation Loss: 0.006790733602311876
Epoch 4 	Training Loss: 0.24978592566949684 	 Validation Loss: 0.006352458971518057
Validation Loss Decreased(4.291987--->4.287910) 	 Saving The Model
Epoch 5 	Training Loss: 0.24845119293062254 	 Validation Loss: 0.007173640551390472
Epoch 6 	Training Loss: 0.2470862954457159 	 Validation Loss: 0.006688327436093931
Epoch 7 	Training Loss: 0.24585621176625252 	 Validation Loss: 0.006713459050213849
Epoch 8 	Training Loss: 0.24481407304052338 	 Validation Loss: 0.006564212198610659
Epoch 9 	Training Loss: 0.24386947556451644 	 Validation Loss: 0.006266007776613589
Validation Loss Decreased(4.287910--->4.229555) 	 Saving The Model
Epoch 10 	Training Loss: 0.2431443692314479 	 Val

Epoch 87 	Training Loss: 0.22280522382586454 	 Validation Loss: 0.005935592386457655
Epoch 88 	Training Loss: 0.22274352231642738 	 Validation Loss: 0.005199275281694201
Validation Loss Decreased(3.586026--->3.509511) 	 Saving The Model
Epoch 89 	Training Loss: 0.2228217920109569 	 Validation Loss: 0.005712532467312283
Epoch 90 	Training Loss: 0.2226800255734345 	 Validation Loss: 0.005443256784368444
Epoch 91 	Training Loss: 0.22266982395780674 	 Validation Loss: 0.00553045528906363
Epoch 92 	Training Loss: 0.2223323428568948 	 Validation Loss: 0.006056031650967068
Epoch 93 	Training Loss: 0.22236619671495217 	 Validation Loss: 0.006067947193428322
Epoch 94 	Training Loss: 0.22227629085448128 	 Validation Loss: 0.005265982415941026
Epoch 95 	Training Loss: 0.2222099313426115 	 Validation Loss: 0.005745940738254123
Epoch 96 	Training Loss: 0.22208116057960467 	 Validation Loss: 0.0055302989041363755
Epoch 97 	Training Loss: 0.2218000528478251 	 Validation Loss: 0.005577346660472729
Epo

Epoch 182 	Training Loss: 0.21617260260364501 	 Validation Loss: 0.005869694992348
Epoch 183 	Training Loss: 0.21643355246225815 	 Validation Loss: 0.005592821968926324
Epoch 184 	Training Loss: 0.21653371522790818 	 Validation Loss: 0.00580335502271299
Epoch 185 	Training Loss: 0.21651750397452346 	 Validation Loss: 0.00647013955646091
Epoch 186 	Training Loss: 0.21612127908720632 	 Validation Loss: 0.005781848077420835
Epoch 187 	Training Loss: 0.21619293125726843 	 Validation Loss: 0.005519756829297101
Epoch 188 	Training Loss: 0.21658709767375736 	 Validation Loss: 0.005549701143194128
Epoch 189 	Training Loss: 0.2161024077748828 	 Validation Loss: 0.005489676528506808
Epoch 190 	Training Loss: 0.21597313610743626 	 Validation Loss: 0.00608207314102738
Epoch 191 	Training Loss: 0.21587186061677377 	 Validation Loss: 0.0053812041106047455
Epoch 192 	Training Loss: 0.2159548663555979 	 Validation Loss: 0.005737411269435176
Epoch 193 	Training Loss: 0.2161144750468325 	 Validation Los

Epoch 278 	Training Loss: 0.2133641489344612 	 Validation Loss: 0.005941880544026693
Epoch 279 	Training Loss: 0.21357163775837948 	 Validation Loss: 0.0058289811346266
Epoch 280 	Training Loss: 0.2139575784742103 	 Validation Loss: 0.006360960006713867
Epoch 281 	Training Loss: 0.21347089901339977 	 Validation Loss: 0.005758455859290229
Epoch 282 	Training Loss: 0.21361298357441463 	 Validation Loss: 0.00562068259274518
Epoch 283 	Training Loss: 0.21347165608299984 	 Validation Loss: 0.005830219233477557
Epoch 284 	Training Loss: 0.2136348159137725 	 Validation Loss: 0.005934748031474926
Epoch 285 	Training Loss: 0.21348670307335518 	 Validation Loss: 0.005690726968977187
Epoch 286 	Training Loss: 0.21338808555867347 	 Validation Loss: 0.0051392784825077765
Epoch 287 	Training Loss: 0.2140557195108645 	 Validation Loss: 0.005479973863672327
Epoch 288 	Training Loss: 0.21361725958886746 	 Validation Loss: 0.005526928901672363
Epoch 289 	Training Loss: 0.21341278484939954 	 Validation L

Epoch 374 	Training Loss: 0.21219980863120144 	 Validation Loss: 0.005484857647507279
Epoch 375 	Training Loss: 0.21206107995312967 	 Validation Loss: 0.005746060035846851
Epoch 376 	Training Loss: 0.21216396517805758 	 Validation Loss: 0.005425626083656594
Epoch 377 	Training Loss: 0.21293093809429078 	 Validation Loss: 0.005358303299656621
Epoch 378 	Training Loss: 0.2122385224944183 	 Validation Loss: 0.005876946184370253
Epoch 379 	Training Loss: 0.21238317354148875 	 Validation Loss: 0.006024899924242938
Epoch 380 	Training Loss: 0.21195134607274577 	 Validation Loss: 0.005353026390075684
Epoch 381 	Training Loss: 0.21225769507319212 	 Validation Loss: 0.005306971338060167
Epoch 382 	Training Loss: 0.21190759741822393 	 Validation Loss: 0.005661406958544696
Epoch 383 	Training Loss: 0.21195385564487423 	 Validation Loss: 0.00579056112854569
Epoch 384 	Training Loss: 0.2121852676891423 	 Validation Loss: 0.005458079355734366
Epoch 385 	Training Loss: 0.21205497675906476 	 Validatio

Epoch 470 	Training Loss: 0.2114697547369106 	 Validation Loss: 0.0055615264398080335
Epoch 471 	Training Loss: 0.21147024969083095 	 Validation Loss: 0.005649124251471626
Epoch 472 	Training Loss: 0.21146143648551047 	 Validation Loss: 0.006110883818732368
Epoch 473 	Training Loss: 0.2111009944300763 	 Validation Loss: 0.0056568742681432655
Epoch 474 	Training Loss: 0.21113127435208429 	 Validation Loss: 0.006109002254627369
Epoch 475 	Training Loss: 0.21149423548279755 	 Validation Loss: 0.005869077488228127
Epoch 476 	Training Loss: 0.2115605406664812 	 Validation Loss: 0.005488821047323722
Epoch 477 	Training Loss: 0.21085295537510845 	 Validation Loss: 0.006019838121202257
Epoch 478 	Training Loss: 0.21114590938085975 	 Validation Loss: 0.005841683546702067
Epoch 479 	Training Loss: 0.21130198592780738 	 Validation Loss: 0.0062023121339303475
Epoch 480 	Training Loss: 0.2110249961266749 	 Validation Loss: 0.0057652273884526
Epoch 481 	Training Loss: 0.21140988505920924 	 Validatio

Epoch 566 	Training Loss: 0.2104875252650851 	 Validation Loss: 0.00585927097885697
Epoch 567 	Training Loss: 0.21081605534841716 	 Validation Loss: 0.00585175461239285
Epoch 568 	Training Loss: 0.2106252406586706 	 Validation Loss: 0.007071437747390182
Epoch 569 	Training Loss: 0.21081520718669822 	 Validation Loss: 0.0061087630413196705
Epoch 570 	Training Loss: 0.21123834352140564 	 Validation Loss: 0.005689276236074942
Epoch 571 	Training Loss: 0.21082662568076435 	 Validation Loss: 0.005675036553983335
Epoch 572 	Training Loss: 0.21040691063498673 	 Validation Loss: 0.006062769801528365
Epoch 573 	Training Loss: 0.21041906902668076 	 Validation Loss: 0.005846739168520327
Epoch 574 	Training Loss: 0.21081170198273827 	 Validation Loss: 0.005804416338602702
Epoch 575 	Training Loss: 0.21076374894652403 	 Validation Loss: 0.006297594441307916
Epoch 576 	Training Loss: 0.21071155473705924 	 Validation Loss: 0.005400239185050682
Epoch 577 	Training Loss: 0.21069688991701158 	 Validatio

Epoch 662 	Training Loss: 0.21094502311947172 	 Validation Loss: 0.005694498132776331
Epoch 663 	Training Loss: 0.21276234216300213 	 Validation Loss: 0.00583719845171328
Epoch 664 	Training Loss: 0.21203005306138167 	 Validation Loss: 0.005985128702940764
Epoch 665 	Training Loss: 0.21062662595835357 	 Validation Loss: 0.005771128601498074
Epoch 666 	Training Loss: 0.21140740314566916 	 Validation Loss: 0.006046065047935203
Epoch 667 	Training Loss: 0.20999245959982238 	 Validation Loss: 0.005867114950109411
Epoch 668 	Training Loss: 0.21134382686351555 	 Validation Loss: 0.00563866933186849
Epoch 669 	Training Loss: 0.21064061585580326 	 Validation Loss: 0.005811728106604682
Epoch 670 	Training Loss: 0.21077907266641574 	 Validation Loss: 0.005924431571254024
Epoch 671 	Training Loss: 0.21120195448243886 	 Validation Loss: 0.005890352637679489
Epoch 672 	Training Loss: 0.21071314307266048 	 Validation Loss: 0.00637682905903569
Epoch 673 	Training Loss: 0.21039492686921063 	 Validatio

Epoch 758 	Training Loss: 0.2116313391773622 	 Validation Loss: 0.00542369630601671
Epoch 759 	Training Loss: 0.21138876015206465 	 Validation Loss: 0.005941112836201985
Epoch 760 	Training Loss: 0.21231753237940534 	 Validation Loss: 0.005968241603286178
Epoch 761 	Training Loss: 0.21124286730288575 	 Validation Loss: 0.006057615280151367
Epoch 762 	Training Loss: 0.21397120250036064 	 Validation Loss: 0.0056798208201373065
Epoch 763 	Training Loss: 0.2137120984022291 	 Validation Loss: 0.006155205037858751
Epoch 764 	Training Loss: 0.21218701010838587 	 Validation Loss: 0.006298132825780798
Epoch 765 	Training Loss: 0.2120003850041472 	 Validation Loss: 0.0064812193976508245
Epoch 766 	Training Loss: 0.21176835686288853 	 Validation Loss: 0.006633173624674479
Epoch 767 	Training Loss: 0.21178585602523753 	 Validation Loss: 0.006868056367944788
Epoch 768 	Training Loss: 0.21142578871439333 	 Validation Loss: 0.006610781528331615
Epoch 769 	Training Loss: 0.21226014796344714 	 Validati

In [ ]:
# evaluate the model
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)
# make a single prediction (expect class=1)
row = [1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
yhat = predict(row, model)
print('Predicted: %.3f (class=%d)' % (yhat, yhat.round()))